# Multiple Feature Linear Regression

---

Author: JGStrickland

Date: June 2025

Description: This notebook demonstrates how to build a linear regression model from scratch using Python and Numpy, without relying on machine learning libraries.

It covers the following steps:
1. Class Definition – Creating a reusable Linear Regression class for multiple features.
2. Fitting the Model – Using the Normal Equation to calculate the optimal slopes and intercept.
3. Making Predictions – Predicting new data points with the trained model.
4. Evaluation – Measuring model performance using MAE, MSE, and R² score.
5. Testing - Testing model performance on real world datasets.

This notebook is intended as a hands-on introduction to linear regression and demonstrates how the underlying mathematics can be implemented programmatically.


## Step Zero - Imports

---
We will use Numpy to handle the matrix multiplication needed in this project. Numpy is preferred over native Python lists because it is much faster, more memory-efficient, and provides built-in support for linear algebra operations like matrix multiplication, transposes, and inverses.

In [77]:
import numpy as np

## Step One - Creating the class

---

First, we need to create a class for the Linear Regression Model so we can have multiple instances of our model to test across different data sets. Here we will call it LinearRegressor. This class will contain three attributes:

- coef_ => this is the coefficient or the slope(s) for features
- intercept_ => this is our y intercept or the bias term
- fitted => this is a boolean value to keep trac wether the model is fitted to the data or not, this starts off as false as our model is not immediately fitted to data

In [78]:
class LinearRegressionMulti:
    def __init__(self):
        self.coef_ = None       # slope(s) for features
        self.intercept_ = None  # bias term
        self.fitted = False

## Step Two - Fitting the data

---

### Understanding the concept of linear regression

In linear regression, we try to fit a straight line through our data points that best predicts the target $y$ from one or more features $X$. The line is generally written as:


$$
y = mX + b
$$

Where:
- $\hat{y}$ is the predicted value of y.
- $m$(slope) determines how much the prediction changes for a unit change in the feature.
- - Example: if $m = 2$, then for every 1 unit increase in $X$, $\hat{y}$ increases by 2.
- $b$ (intercept or bias) determined where the line crosses the $y$-axis.
- - The intercept is the starting point of the line when all features are zero. It shifts the line up or down so that the model can best fit the data instead of being forced through the origin.

for multiple features, this generalises to:
$$
\hat{y} = \beta_{0} + \beta_{1}X_{1} + \beta_{2}X_{2} + ... \beta_{n}X_{n}
$$
where:
- $\beta_{0}$ = intercept/bias.
- $\beta_{1},...\beta_{n}$ = slopes for each fetaure.

The goal of linear regression is to find the slopes and intercept the makes the predictions $\hat{y}$ as close as possible to the actual values of $y$.

---

### Cost function: Measuring error

To measure how accurate/ far off our predictions are we define a cost/loss function $J\left(\beta\right) as the sum of squared errors:

$$
J\left(\beta\right) = \left\|y - \hat{y} \right\|^{2}
$$

In matrix form where $\hat{y} = X\beta$:

$$
J\left(\beta\right) = \left(y - X\beta\right)^{T} \left(y - X\beta\right)
$$

---
### Adding bias

Instead of calculating both the slopes and intercept seperately we can add a column of ones to $X$ to represent the bias. This lets the model learn the intercept and the slopes all at once. This helps the model generalise to any number of features.

---

### Solving the parameters: The Normal Equation

To find the best $\beta$ that minimises $J\left(\beta\right)$, we use the Normal Equation:

$$
\beta = \left(X^{T}X\right)^{-1} X^{T}y
$$

- This gives all slopes and the intercept in a single computation.
- The slopes tell us how strongly each feature affects the prediction.
- The intercept allows the line to shift up or down so it fits the data properly.

Using the Normal Equation, we find the combination of parameters that minimizes the squared error, giving the best linear approximation of the data.

In [79]:
class LinearRegressionMulti:
    def __init__(self):
        self.coef_ = None       # slope(s) for features
        self.intercept_ = None  # bias term
        self.fitted = False

    def fit(self, X, y):
        """Fit model using Normal Equation"""
        # Convert to numpy arrays
        X = np.asarray(X)
        y = np.asarray(y)

        # Ensure y is a column vector
        if y.ndim == 1:
            y = y.reshape(-1, 1)

        # Add bias (column of 1s) to X
        X_b = np.c_[np.ones((X.shape[0], 1)), X]

        # Normal Equation: (XᵀX)^(-1) Xᵀy
        theta = np.linalg.pinv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)

        # Extract intercept and coefficients
        self.intercept_ = theta[0][0]
        self.coef_ = theta[1:].flatten()
        self.fitted = True

## Step Three - Making the Predictions

---

The predict function uses the trained model parameters (slopes and intercept) to calculate predictions for new data. It takes the feature matrix X, applies the linear equation $\hat{y} = X\beta$, and returns the predicted values. If X has only one feature, it automatically reshapes it into a column vector so that the matrix multiplication works correctly.


In [80]:
class LinearRegressionMulti:
    def __init__(self):
        self.coef_ = None       # slope(s) for features
        self.intercept_ = None  # bias term
        self.fitted = False

    def fit(self, X, y):
        """Fit model using Normal Equation"""
        # Convert to numpy arrays
        X = np.asarray(X)
        y = np.asarray(y)

        # Ensure y is a column vector
        if y.ndim == 1:
            y = y.reshape(-1, 1)

        # Add bias (column of 1s) to X
        X_b = np.c_[np.ones((X.shape[0], 1)), X]

        # Normal Equation: (XᵀX)^(-1) Xᵀy
        theta = np.linalg.pinv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)

        # Extract intercept and coefficients
        self.intercept_ = theta[0][0]
        self.coef_ = theta[1:].flatten()
        self.fitted = True

    def predict(self, X):
        """Predict values given new data"""
        if not self.fitted:
            raise ValueError("Model not fitted yet. Call fit() first.")

        X = np.asarray(X)

        # If X is 1D (single feature), reshape it
        if X.ndim == 1:
            X = X.reshape(-1, 1)

        return X.dot(self.coef_) + self.intercept_

## Step Four - Evaluating our Predictions

---

Once our model is trained, we need a way to quantify how well it is performing. Evaluating the model is crucial to understand if it is making accurate predictions or if it needs improvement. In linear regression, there are several commonly used metrics:

### 1. Mean Absolute Error (MAE)

The Mean Absolute Error is the average of the absolute differences between the actual values and the predicted values:

$$
MAE = \frac{1}{n} \sum_{i=1}^{n} \left| y_i - \hat{y}_i \right|
$$

- Interpretation: MAE gives an intuitive measure of how far off our predictions are, on average. Lower values indicate better predictive accuracy.
- Pros: Simple to understand and not sensitive to outliers as much as MSE.
- Cons: It does not penalize large errors as strongly as the Mean Squared Error does.

---

### 2. Mean Squared Error (MSE)

The Mean Squared Error is the average of the squared differences between the actual and predicted values:

$$
MSE = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2
$$

- Interpretation: Squaring the errors emphasizes larger errors more than smaller ones, which makes MSE sensitive to outliers.
- Pros: Provides a stronger penalty for large errors and is differentiable, which is useful for optimization.
- Cons: Harder to interpret because it is in squared units of the target variable.

---

### 3. R² Score (Coefficient of Determination)

The R² score measures the proportion of the variance in the dependent variable that is predictable from the independent variables:

$$
R^2 = 1 - \frac{\sum_{i=1}^{n} (y_i - \hat{y}_i)^2}{\sum_{i=1}^{n} (y_i - \bar{y})^2}
$$

- Interpretation: R² ranges from 0 to 1, where 1 indicates perfect predictions and 0 indicates that the model does no better than simply predicting the mean of the target variable.
- Pros: Provides a normalized measure of how well the model explains the variance in the data.
- Cons: Can be misleading for models that overfit or when comparing models across different datasets.

---

By calculating these metrics, we can objectively assess our model’s performance and identify areas for improvement. Typically, a combination of MAE, MSE, and R² is used to get a complete picture of predictive accuracy and error distribution.

In [81]:
class LinearRegressionMulti:
    def __init__(self):
        self.coef_ = None       # slope(s) for features
        self.intercept_ = None  # bias term
        self.fitted = False

    def fit(self, X, y):
        """Fit model using Normal Equation"""
        # Convert to numpy arrays
        X = np.asarray(X)
        y = np.asarray(y)

        # Ensure y is a column vector
        if y.ndim == 1:
            y = y.reshape(-1, 1)

        # Add bias (column of 1s) to X
        X_b = np.c_[np.ones((X.shape[0], 1)), X]

        # Normal Equation: (XᵀX)^(-1) Xᵀy
        theta = np.linalg.pinv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)

        # Extract intercept and coefficients
        self.intercept_ = theta[0][0]
        self.coef_ = theta[1:].flatten()
        self.fitted = True

    def predict(self, X):
        """Predict values given new data"""
        if not self.fitted:
            raise ValueError("Model not fitted yet. Call fit() first.")

        X = np.asarray(X)

        # If X is 1D (single feature), reshape it
        if X.ndim == 1:
            X = X.reshape(-1, 1)

        return X.dot(self.coef_) + self.intercept_

    # ---------------- Evaluation Metrics ---------------- #

    def mae_score(self, y_true, y_pred):
        """Mean Absolute Error"""
        return np.mean(np.abs(y_true - y_pred))

    def mse_score(self, y_true, y_pred):
        """Mean Squared Error"""
        return np.mean((y_true - y_pred) ** 2)

    def r2_score(self, y_true, y_pred):
        """R² score (variance explained)"""
        mean_y = np.mean(y_true)
        ss_total = np.sum((y_true - mean_y) ** 2)
        ss_residual = np.sum((y_true - y_pred) ** 2)
        return 1 - (ss_residual / ss_total) if ss_total != 0 else 0

## Step Five - Testing Our Model

---

Now that we have a working linear regression model, we can test it on real-world datasets to evaluate its performance. We will use two datasets from `scikit-learn`:

1. California Housing Dataset
   - Predict median house values based on 8 features like income, population, and housing density.
   - A larger dataset with real-world complexity.

2. Diabetes Dataset
   - Predict a quantitative measure of disease progression after one year using 10 clinical features.
   - Smaller and simpler, ideal for testing and debugging.

We will:
- Load the dataset
- Split it into features (`X`) and target (`y`)
- Fit our model
- Make predictions
- Evaluate performance using MAE, MSE, and R² score

In [82]:
from sklearn.datasets import fetch_california_housing, load_diabetes
from sklearn.model_selection import train_test_split

# Initialize our Linear Regression model
model = LinearRegressionMulti()

# ----------------- Test on California Housing ----------------- #
# Load dataset
california = fetch_california_housing()
X_cal, y_cal = california.data, california.target

# Split into training and testing sets
X_train_cal, X_test_cal, y_train_cal, y_test_cal = train_test_split(X_cal, y_cal, test_size=0.2, random_state=42)

# Fit the model
model.fit(X_train_cal, y_train_cal)

# Predict
y_pred_cal = model.predict(X_test_cal)

# Evaluate
print("California Housing Dataset")
print("---------------------------")
print("MAE:", model.mae_score(y_test_cal, y_pred_cal))
print("MSE:", model.mse_score(y_test_cal, y_pred_cal))
print("R²:", model.r2_score(y_test_cal, y_pred_cal))
print("\n")

# ----------------- Test on Diabetes Dataset ----------------- #
# Load dataset
diabetes = load_diabetes()
X_dia, y_dia = diabetes.data, diabetes.target

# Split into training and testing sets
X_train_dia, X_test_dia, y_train_dia, y_test_dia = train_test_split(X_dia, y_dia, test_size=0.2, random_state=36)

# Fit the model
model.fit(X_train_dia, y_train_dia)

# Predict
y_pred_dia = model.predict(X_test_dia)

# Evaluate
print("Diabetes Dataset")
print("----------------")
print("MAE:", model.mae_score(y_test_dia, y_pred_dia))
print("MSE:", model.mse_score(y_test_dia, y_pred_dia))
print("R²:", model.r2_score(y_test_dia, y_pred_dia))


California Housing Dataset
---------------------------
MAE: 0.5332001305418295
MSE: 0.5558915987280281
R²: 0.5757877060074328


Diabetes Dataset
----------------
MAE: 44.817857530800275
MSE: 3013.5452613146817
R²: 0.40748760654298455


## Conclusion

---

In this notebook, we implemented a Linear Regression model from scratch capable of handling multiple features. This exercise illustrates the core principles of linear regression, including fitting parameters using the Normal Equation, making predictions, and evaluating model performance with metrics such as MAE, MSE, and R².

### Strengths of Linear Regression
- Simplicity and interpretability: Linear regression is easy to understand and provides direct insight into how each feature affects the target variable.
- Efficiency: Using the Normal Equation allows for a closed-form solution without iterative optimization.
- Foundation for more complex models: It serves as the basis for many advanced regression and machine learning methods.

### Weaknesses of Linear Regression
- Limited to linear relationships: It assumes that the relationship between features and target is linear, which may not hold in real-world data.
- Sensitive to outliers: Extreme values can disproportionately influence the slope and intercept.
- Feature scaling may be necessary: Variables on very different scales can affect model performance, especially in gradient-based variants.

### Analysis of Results
- California Housing Dataset: The model achieved an R² of approximately 0.58, indicating that it explains over half of the variance in median house prices. While this is a reasonable result for a simple linear model, the moderate R² suggests that some non-linear relationships and complex interactions between features are not captured.
- Diabetes Dataset: The R² of around 0.45 reflects a modest fit, highlighting that linear regression can capture general trends but may struggle with more subtle or non-linear effects present in clinical data. The MAE and MSE values confirm that the predictions are reasonably close to actual values but leave room for improvement.

Overall, this notebook demonstrates that linear regression is a powerful and interpretable tool for baseline modeling, but its assumptions and limitations must be considered when applying it to real-world datasets. Further improvements could include feature scaling, polynomial features, regularization, or more sophisticated algorithms to better capture complex patterns.


